In [1]:
%load_ext autoreload
%autoreload 2
import os
import datetime
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from pyspark.sql import functions as F
from pyspark.sql.types import *

from distribution_cost.configuration import spark_config
from distribution_cost.configuration.app import AppConfig
from distribution_cost.configuration.data import DataConfig
from distribution_cost.infra import oracle
from distribution_cost.domain import kpis

import numpy as np

/gpfs/user/e587246/dco00/conf/application.yml
/gpfs/user/e587246/dco00


In [2]:
# Database uri
app_config = AppConfig()

db_uri = app_config.db_uri_jdbc

In [ ]:
# Create spark session
spark_context, spark_session = spark_config.get_spark(app_name="app-distribution-cost",
                                                      executors=4, executor_cores=4, executor_mem='16g',
                                                      dynamic_allocation=True, max_executors=8)

spark_session.conf.set("spark.sql.crossJoin.enabled", "true")

In [ ]:
df_sinqtvin = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtvin/")
df_sinqtcli = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtcli/")
df_sinqtver = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtver/")
df_sinqtfv4 = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtfv4/")
df_sinqtcmp = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtcmp/")
df_sinqtcnd = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtcnd/")
df_sinqtseg = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtseg/")
df_sinqtzds = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtzds/")
df_sinqtsfa = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtsfa/")
df_sinqtfam = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtfam/")
df_sinqtrub = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtrub/")
df_sinqtopc = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtopc/")
df_sinqtcma = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtcma/")
df_sinqtcmi = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtcmi/")
df_sinqtcyr = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtcyr/")
df_sinqtmrq = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtmrq/")
df_sinqtbas = spark_session.read.load("/user/brc10/data/standardized/sinc0/sinqtbas/")

In [ ]:
df_sinqtvin.createOrReplaceTempView("df_sinqtvin")
df_sinqtver.createOrReplaceTempView("df_sinqtver")
df_sinqtfv4.createOrReplaceTempView("df_sinqtfv4")
df_sinqtcmp.createOrReplaceTempView("df_sinqtcmp")
df_sinqtcnd.createOrReplaceTempView("df_sinqtcnd")
df_sinqtcli.createOrReplaceTempView("df_sinqtcli")
df_sinqtseg.createOrReplaceTempView("df_sinqtseg")
df_sinqtzds.createOrReplaceTempView("df_sinqtzds")
df_sinqtsfa.createOrReplaceTempView("df_sinqtsfa")
df_sinqtfam.createOrReplaceTempView("df_sinqtfam")
df_sinqtrub.createOrReplaceTempView("df_sinqtrub")
df_sinqtopc.createOrReplaceTempView("df_sinqtopc")
df_sinqtcma.createOrReplaceTempView("df_sinqtcma")
df_sinqtcmi.createOrReplaceTempView("df_sinqtcmi")
df_sinqtcyr.createOrReplaceTempView("df_sinqtcyr")
df_sinqtmrq.createOrReplaceTempView("df_sinqtmrq")
df_sinqtbas.createOrReplaceTempView("df_sinqtbas")

In [ ]:
querySAMARA = """
SELECT
    SINQTVIN.CODE SINQTVIN__CODE,
    SINQTCLI_2.CODE SINQTCLI_2__CODE,
    SINQTCLI_2.CODE_PAYS_IMPLANT,
    SINQTVER.CODE SINQTVER__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTVER.LIB_EN,SINQTVER.LIB_FR) 
when 'fr_FR' then SINQTVER.LIB_FR
when 'es_SP' then nvl(SINQTVER.LIB_ES,SINQTVER.LIB_FR)
else SINQTVER.LIB_FR

end SINQTVER__LIB,
    Table__54.DT_FACT,
    Table__54.DT_VD,
    Table__54.DT_COMM_CLI_FIN_VD,
    Table__54.DATIMM,
    SINQTCMP.CODE SINQTCMP__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTCMP.LIB_EN,SINQTCMP.LIB_FR)
when 'fr_FR' then SINQTCMP.LIB_FR
when 'es_SP' then nvl(SINQTCMP.LIB_ES,SINQTCMP.LIB_FR)
else SINQTCMP.LIB_FR

end SINQTCMP__LIB,
    SINQTCND.CODE SINQTCND__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTCND.LIB_EN,SINQTCND.LIB_FR)
when 'fr_FR' then SINQTCND.LIB_FR
when 'es_SP' then nvl(SINQTCND.LIB_ES,SINQTCND.LIB_FR)
else SINQTCND.LIB_FR

end SINQTCND__LIB,
    SINQTCLI.CODE SINQTCLI__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTCLI.LIB_EN,SINQTCLI.LIB_FR)
when 'fr_FR' then SINQTCLI.LIB_FR
when 'es_SP' then nvl(SINQTCLI.LIB_ES,SINQTCLI.LIB_FR)
else SINQTCLI.LIB_FR

end SINQTCLI__LIB,
    SINQTSEG.CODE SINQTSEG__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTSEG.LIB_EN,SINQTSEG.LIB_FR)
when 'fr_FR' then SINQTSEG.LIB_FR
when 'es_SP' then nvl(SINQTSEG.LIB_ES,SINQTSEG.LIB_FR)
else SINQTSEG.LIB_FR

end SINQTSEG__LIB,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTZDS.LIB_EN,SINQTZDS.LIB_FR)
when 'fr_FR' then SINQTZDS.LIB_FR
when 'es_SP' then nvl(SINQTZDS.LIB_ES,SINQTZDS.LIB_FR)
else SINQTZDS.LIB_FR

end SINQTZDS__LIB,
    SINQTSFA.CODE SINQTSFA__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTSFA.LIB_EN,SINQTSFA.LIB_FR)
when 'fr_FR' then SINQTSFA.LIB_FR
when 'es_SP' then nvl(SINQTSFA.LIB_ES,SINQTSFA.LIB_FR)
else SINQTSFA.LIB_FR

end SINQTSFA__LIB,
    SINQTFAM.CODE SINQTFAM__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTFAM.LIB_EN,SINQTFAM.LIB_FR)
when 'fr_FR' then SINQTFAM.LIB_FR
when 'es_SP' then nvl(SINQTFAM.LIB_ES,SINQTFAM.LIB_FR)
else SINQTFAM.LIB_FR

end SINQTFAM__LIB,
SINQTRUB.CODE SINQTRUB__CODE,
case 'fr_FR'
when 'en_GB' then SINQTRUB.LIB_EN
when 'fr_FR' then SINQTRUB.LIB_FR
when 'es_SP' then SINQTRUB.LIB_ES
else SINQTRUB.LIB_FR

end SINQTRUB__LIB,
    SINQTOPC.CODE SINQTOPC__CODE,
    case 'fr_FR'
when 'en_GB' then SINQTOPC.LIB_EN
when 'fr_FR' then SINQTOPC.LIB_FR
when 'es_SP' then SINQTOPC.LIB_ES
else SINQTOPC.LIB_FR

end SINQTOPC_LIB,
    SINQTCMA.CODE SINQTCMA__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTCMA.LIB_EN,SINQTCMA.LIB_FR)
when 'fr_FR' then SINQTCMA.LIB_FR
when 'es_SP' then nvl(SINQTCMA.LIB_ES,SINQTCMA.LIB_FR)
else SINQTCMA.LIB_FR

end SINQTCMA__LIB,
    SINQTCMI.CODE SINQTCMI__CODE,
    case 'fr_FR'
when 'en_GB' then nvl(SINQTCMI.LIB_EN,SINQTCMI.LIB_FR)
when 'fr_FR' then SINQTCMI.LIB_FR
when 'es_SP' then nvl(SINQTCMI.LIB_ES,SINQTCMI.LIB_FR)
else SINQTCMI.LIB_FR

end SINQTCMI__LIB,
    Table__54.TYPE_FLOTTE_VD,
    Table__54.TYPE_OPE_ESSOR,
    Table__54.TYP_UTIL_VD,
    Table__54.CODE_PROFESSION_VD,
    Table__54.CODE_PROMO,
    Table__54.CODE_PROMO2,
    SINQTCYR.ANNEE_MOIS,
    sum(Table__54.VOLUME_AJ) VOLUME_AJ,
    sum(Table__54.PRIX_VENTE) PRIX_VENTE,
    sum(Table__54.PRIX_VENTE_AJ) PRIX_VENTE_AJ,
    sum(Table__54.PV_OPTIONS) PV_OPTIONS,
    sum(Table__54.PV_VERSION) PV_VERSION,
    sum(Table__54.MACOM_CONSO) MACOM_CONSO,
    sum(Table__54.MACOM_CONSO_AJ) MACOM_CONSO_AJ,
    sum(Table__54.MACOM_CONSO_VERSION) MACOM_CONSO_VERSION,
    sum(Table__54.MACOM_CONSO_OPTION) MACOM_CONSO_OPTION,
    sum(Table__54.MACOM_ENTITE) MACOM_ENTITE,
    sum(Table__54.MACOM_ENTITE_AJ) MACOM_ENTITE_AJ,
    sum(Table__54.MACOM_ENTITE_VERSION) MACOM_ENTITE_VERSION,
    sum(Table__54.MACOM_ENTITE_OPTION) MACOM_ENTITE_OPTION,
    sum(Table__54.RBCV_AJ) RBCV_AJ,
    sum(Table__54.MCX_VARIABLES) MCX_VARIABLES,
    SINQTMRQ_2.CODE SINQTMRQ_2__CODE
FROM
    df_sinqtfv4 TABLE__54,
    df_sinqtfam SINQTFAM_2
    
JOIN df_sinqtcmp SINQTCMP ON Table__54.ID_ZDS=SINQTCMP.ID_ZDS and Table__54.ID_CMP=SINQTCMP.ID
JOIN df_sinqtcli SINQTCLI ON Table__54.ID_ZDS=SINQTCLI.ID_ZDS and Table__54.ID_SCD=SINQTCLI.ID_SCD and Table__54.ID_CLI=SINQTCLI.ID
JOIN df_sinqtcmi SINQTCMI ON Table__54.ID_ZDS=SINQTCMI.ID_ZDS and Table__54.ID_CMI=SINQTCMI.ID
JOIN df_sinqtbas SINQTBAS ON Table__54.ID_BAS=SINQTBAS.ID
JOIN df_sinqtzds SINQTZDS ON Table__54.ID_ZDS=SINQTZDS.ID
LEFT OUTER JOIN df_sinqtrub SINQTRUB ON Table__54.ID_RUB=SINQTRUB.ID
JOIN df_sinqtfam SINQTFAM ON Table__54.ID_ZDS=SINQTFAM.ID_ZDS and Table__54.ID_FAM=SINQTFAM.ID
JOIN df_sinqtsfa SINQTSFA ON Table__54.ID_ZDS=SINQTSFA.ID_ZDS and Table__54.ID_SFA=SINQTSFA.ID
JOIN df_sinqtver SINQTVER ON Table__54.ID_ZDS=SINQTVER.ID_ZDS and Table__54.ID_VER=SINQTVER.ID
LEFT OUTER JOIN df_sinqtseg SINQTSEG ON Table__54.ID_ZDS=SINQTSEG.ID_ZDS and Table__54.ID_SEG=SINQTSEG.ID
JOIN df_sinqtcnd SINQTCND ON Table__54.ID_CND=SINQTCND.ID
JOIN df_sinqtcma SINQTCMA ON Table__54.ID_CMA=SINQTCMA.ID
LEFT OUTER JOIN df_sinqtopc SINQTOPC ON Table__54.ID_ZDS=SINQTOPC.ID_ZDS and Table__54.ID_OPC=SINQTOPC.ID
JOIN df_sinqtvin SINQTVIN ON Table__54.ID_ZDS=SINQTVIN.ID_ZDS and Table__54.ID_VIN=SINQTVIN.ID
JOIN df_sinqtcyr SINQTCYR ON Table__54.ID_CYC=SINQTCYR.ID
JOIN df_sinqtcli SINQTCLI_2 ON Table__54.ID_ZDS=SINQTCLI_2.ID_ZDS and Table__54.ID_SCD=SINQTCLI_2.ID_SCD and Table__54.ID_CLI_LIV=SINQTCLI_2.ID
JOIN df_sinqtmrq SINQTMRQ_2 ON SINQTFAM_2.ID_MRQ_COM=SINQTMRQ_2.ID and Table__54.ID_ZDS=SINQTFAM_2.ID_ZDS and Table__54.ID_FAM=SINQTFAM_2.ID    

WHERE   ( Table__54.DT_VD BETWEEN TO_DATE('01/01/2019', 'dd/MM/yyyy') AND TO_DATE('30/06/2019', 'dd/MM/yyyy'))
    AND ( SINQTBAS.CODE  =  'LA' AND SINQTCLI_2.CODE_PAYS_IMPLANT  IN  ( 'FR','DE','PT','BE','IT','ES','GB','NL','PL','AT'  ) AND ( SINQTBAS.CODE != 'EA'  ))
GROUP BY
  SINQTVIN.CODE, 
  SINQTCLI_2.CODE, 
  SINQTCLI_2.CODE_PAYS_IMPLANT, 
  SINQTVER.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTVER.LIB_EN,SINQTVER.LIB_FR)
when 'fr_FR' then SINQTVER.LIB_FR
when 'es_SP' then nvl(SINQTVER.LIB_ES,SINQTVER.LIB_FR)
else SINQTVER.LIB_FR

end, 
  Table__54.DT_FACT, 
  Table__54.DT_VD, 
  Table__54.DT_COMM_CLI_FIN_VD, 
  Table__54.DATIMM, 
  SINQTCMP.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTCMP.LIB_EN,SINQTCMP.LIB_FR)
when 'fr_FR' then SINQTCMP.LIB_FR
when 'es_SP' then nvl(SINQTCMP.LIB_ES,SINQTCMP.LIB_FR)
else SINQTCMP.LIB_FR

end, 
  SINQTCND.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTCND.LIB_EN,SINQTCND.LIB_FR)
when 'fr_FR' then SINQTCND.LIB_FR
when 'es_SP' then nvl(SINQTCND.LIB_ES,SINQTCND.LIB_FR)
else SINQTCND.LIB_FR

end, 
  SINQTCLI.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTCLI.LIB_EN,SINQTCLI.LIB_FR)
when 'fr_FR' then SINQTCLI.LIB_FR
when 'es_SP' then nvl(SINQTCLI.LIB_ES,SINQTCLI.LIB_FR)
else SINQTCLI.LIB_FR

end, 
  SINQTSEG.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTSEG.LIB_EN,SINQTSEG.LIB_FR)
when 'fr_FR' then SINQTSEG.LIB_FR
when 'es_SP' then nvl(SINQTSEG.LIB_ES,SINQTSEG.LIB_FR)
else SINQTSEG.LIB_FR

end, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTZDS.LIB_EN,SINQTZDS.LIB_FR)
when 'fr_FR' then SINQTZDS.LIB_FR
when 'es_SP' then nvl(SINQTZDS.LIB_ES,SINQTZDS.LIB_FR)
else SINQTZDS.LIB_FR

end, 
  SINQTSFA.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTSFA.LIB_EN,SINQTSFA.LIB_FR)
when 'fr_FR' then SINQTSFA.LIB_FR
when 'es_SP' then nvl(SINQTSFA.LIB_ES,SINQTSFA.LIB_FR)
else SINQTSFA.LIB_FR

end, 
  SINQTFAM.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTFAM.LIB_EN,SINQTFAM.LIB_FR)
when 'fr_FR' then SINQTFAM.LIB_FR
when 'es_SP' then nvl(SINQTFAM.LIB_ES,SINQTFAM.LIB_FR)
else SINQTFAM.LIB_FR

end, 
  SINQTRUB.CODE, 
  case 'fr_FR'
when 'en_GB' then SINQTRUB.LIB_EN
when 'fr_FR' then SINQTRUB.LIB_FR
when 'es_SP' then SINQTRUB.LIB_ES
else SINQTRUB.LIB_FR

end, 
  SINQTOPC.CODE, 
  case 'fr_FR'
when 'en_GB' then SINQTOPC.LIB_EN
when 'fr_FR' then SINQTOPC.LIB_FR
when 'es_SP' then SINQTOPC.LIB_ES
else SINQTOPC.LIB_FR

end, 
  SINQTCMA.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTCMA.LIB_EN,SINQTCMA.LIB_FR)
when 'fr_FR' then SINQTCMA.LIB_FR
when 'es_SP' then nvl(SINQTCMA.LIB_ES,SINQTCMA.LIB_FR)
else SINQTCMA.LIB_FR

end, 
  SINQTCMI.CODE, 
  case 'fr_FR'
when 'en_GB' then nvl(SINQTCMI.LIB_EN,SINQTCMI.LIB_FR)
when 'fr_FR' then SINQTCMI.LIB_FR
when 'es_SP' then nvl(SINQTCMI.LIB_ES,SINQTCMI.LIB_FR)
else SINQTCMI.LIB_FR

end, 
  Table__54.TYPE_FLOTTE_VD, 
  Table__54.TYPE_OPE_ESSOR, 
  Table__54.TYP_UTIL_VD, 
  Table__54.CODE_PROFESSION_VD, 
  Table__54.CODE_PROMO, 
  Table__54.CODE_PROMO2, 
  SINQTCYR.ANNEE_MOIS, 
  SINQTMRQ_2.CODE
  
ORDER BY Table__54.DT_VD
"""

In [ ]:
dfSAMARA = spark_session.sql(querySAMARA).cache()

In [ ]:
dfVINXPROMO= dfSAMARA.select("SINQTVIN__CODE","SINQTZDS__LIB","SINQTOPC_LIB","SINQTRUB__CODE","SINQTRUB__LIB","SINQTOPC__CODE","MCX_VARIABLES").drop_duplicates(['SINQTVIN__CODE','SINQTOPC__CODE'])
dfVINXPROMO.drop_duplicates()

In [ ]:
dfVINXPROMO=dfVINXPROMO.withColumn('Country', F.initcap(F.split(F.col('SINQTZDS__LIB'),' ')[1]))
dfVINXPROMO = dfVINXPROMO.withColumn('vinxpromo', F.concat(F.col('SINQTVIN__CODE'),F.lit('_'), F.col('Country')))

In [ ]:
column_values_rub= {'PROVISION SUR PERTES FUTURES':'PSA - Buy Backs','PRIMES VENTES AUX SOCIETES':'PSA - B2B promotions','PRIME QUALITE':'PSA - Bonus for quality','PRIMES A LA PERFORMANCE RESEAU':'PSA - Bonus for dealer performance','PROMOTIONS CLIENT FINAL':'PSA - B2C promotions' }
dfVINXPROMO = dfVINXPROMO.replace(to_replace=column_values_rub, subset=['SINQTRUB__LIB'])

In [ ]:
dfVINXPROMO = dfVINXPROMO.withColumn('Classification',F.when(F.col('SINQTRUB__LIB').isin('PSA - Bonus for dealer performance','PSA - Bonus for quality'), 'Network Remuneration').otherwise('Client Promotions'))

In [31]:
dfVINXPROMO=dfVINXPROMO.withColumn('SINQTOPC_LIB',F.concat(F.col('SINQTOPC_LIB'),F.lit(' - '), F.col('Classification')))
dfVINXPROMO=dfVINXPROMO.drop('Classification')

In [ ]:
dfVINXPROMO.toPandas().head(10)